## Pain Score evaluation Segment level (Select pose)

In [2]:
from bsoid_kit import *
import pandas as pd
import numpy as np
import os
import joblib

### Feature extract with pose prediction

In [ ]:
root = r'C:\Users\x\Desktop\final\data\old1_landmark7/'
outpath = r'./data/test_output'
landmarknum = 7
framerate = 60
embeder_path = ''#r'./data/model_embed/old_embed'
clf_path = r'./data/model_clf/old_rf'

files = os.listdir(root)
if not os.path.isdir(outpath):
    os.makedirs(outpath)
for file in files:
    filename = root+'/'+file
    savename = (outpath+'/'+file).replace('.csv','')
    if not os.path.isfile(embeder_path):
        feat = bsoidfeat(filename,landmarknum,framerate,savename)
    if os.path.isfile(embeder_path):
        feat = bsoidfeat(filename,landmarknum,framerate)
        embeder = joblib.load(embeder_path)
        clf = joblib.load(clf_path)
        embeddings = embeder.transform(feat)
        motions = motion_predict(feat, embeder, clf)
        joblib.dump([feat,embeddings,motions], savename)
    elif os.path.isfile(clf_path):
        feat = bsoidfeat(filename,landmarknum,framerate)
        clf = joblib.load(clf_path)
        motions = clf.predict(feat)
        joblib.dump([feat,motions], savename)

### pose analysis for Cap data

In [12]:
root = r'./data/new_bsoidfeat/'
files = os.listdir(root)
posenum = 10

poseB = [0]*posenum
poseT = [0]*posenum
for file in files:
    if file.find('Cap')!=-1:
        savfile = joblib.load(root+'/'+file)
        labels = savfile[-1]
        if file.find('basal')!=-1:
            for i in range(posenum):
                poseB[i]+= len(np.where(labels==i)[0])
        if file.find('treat')!=-1:
            for i in range(posenum):
                poseT[i]+= len(np.where(labels==i)[0])


In [13]:
diff = np.array(poseT)-np.array(poseB)
sum2 = np.array(poseT)+np.array(poseB)

In [14]:
print(abs(diff)/sum(abs(diff)))
print(np.array(sum2)/sum(sum2))
print(abs(diff)/sum(abs(diff))>0.04)

[0.00430005 0.03798376 0.01528906 0.02054467 0.00310559 0.16483516
 0.00334448 0.10033445 0.04204491 0.60821787]
[0.04307352 0.04816188 0.08772483 0.0271379  0.09778321 0.03455349
 0.0542758  0.05119912 0.01475229 0.54133796]
[False False False False False  True False  True  True  True]


### Select pose to train classifier

In [6]:
from sklearn.svm import SVC
import joblib
import numpy as np

In [7]:
posenum=10

In [8]:
def select_pose(feat,labels,sel=None):
    sel=np.arange(posenum)
    # sel=[2,4,6,9]
    sel_index = np.array([])
    for s in sel:
        sel_index = np.concatenate([sel_index, np.where(labels==s)[0]])
    sel_index = sel_index.astype(int)
    feat = feat[:,sel_index]
    return feat.T
    

In [9]:
cap_basal_feats = []
cap_treat_feats = []
sng_basal_feats = []
sng_treat_feats = []
ro_basal_feats = []
ro_treat_feats = []
root = r'./data/old1_bsoidfeat/'
files = os.listdir(root)
for file in files:
    [feat,labels] = joblib.load(root+file)
    feat = np.array(feat)
    labels = labels[0]
    if file.find('Cap')!=-1:
        if file.find('basal')!=-1:
            cap_basal_feats.append(select_pose(feat,labels))
        if file.find('treat')!=-1:
            cap_treat_feats.append(select_pose(feat,labels))
    elif file.find('pH5.2')!=-1:
        if file.find('basal')!=-1:
            sng_basal_feats.append(select_pose(feat,labels))
        if file.find('treat')!=-1:
            sng_treat_feats.append(select_pose(feat,labels))
    else:
        if file.find('basal')!=-1:
            ro_basal_feats.append(select_pose(feat,labels))
        if file.find('treat')!=-1:
            ro_treat_feats.append(select_pose(feat,labels))


In [11]:
cap_basal_feats[0].shape

(1556, 49)

In [11]:
x_train = np.zeros([0,49])
y_train = []
x_test = np.zeros([0,49])
y_test = []
bshuffles = []
tshuffles = []
for i in range(len(cap_basal_feats)):
    #basal
    feat = cap_basal_feats[i]
    ##shuffle
    ind = np.arange(len(feat))
    np.random.shuffle(ind)
    bshuffles.append(ind)
    feat = feat[ind,:]
    sp = len(feat)//2
    x_train = np.concatenate([x_train,feat[:sp,:]])
    y_train = y_train + [-1] * len(cap_basal_feats[i][:sp,:])
    x_test = np.concatenate([x_test,cap_basal_feats[i][sp:,:]])
    y_test = y_test + [-1] * len(cap_basal_feats[i][sp:,:])
    feat = cap_treat_feats[i]
    #treat
    feat = cap_treat_feats[i]
    ##shuffle
    ind = np.arange(len(feat))
    np.random.shuffle(ind)
    tshuffles.append(ind)
    feat = feat[ind,:]
    sp = len(feat)//2
    x_train = np.concatenate([x_train,feat[:sp,:]])
    y_train = y_train + [1] * len(feat[:sp,:])
    x_test = np.concatenate([x_test,feat[sp:,:]])
    y_test = y_test + [1] * len(feat[sp:,:])
# add RO
train_split = 10 # 10%
if train_split != 0:
    for i in range(len(ro_basal_feats)):
        sp = len(ro_basal_feats[i])//train_split
        x_train = np.concatenate([x_train,ro_basal_feats[i][:sp,:]])
        y_train = y_train + [-1] * len(ro_basal_feats[i][:sp,:])
        x_test = np.concatenate([x_test,ro_basal_feats[i][sp:,:]])
        y_test = y_test + [-1] * len(ro_basal_feats[i][sp:,:])
        sp = len(ro_treat_feats[i])//train_split
        x_train = np.concatenate([x_train,ro_treat_feats[i][:sp,:]])
        y_train = y_train + [-1] * len(ro_treat_feats[i][:sp,:])
        x_test = np.concatenate([x_test,ro_treat_feats[i][sp:,:]])
        y_test = y_test + [-1] * len(ro_treat_feats[i][sp:,:])
# add Sng
train_split = 10 # 10%
if train_split != 0:
    for i in range(len(sng_basal_feats)):
        sp = len(sng_basal_feats[i])//train_split
        x_train = np.concatenate([x_train,sng_basal_feats[i][:sp,:]])
        y_train = y_train + [-1] * len(sng_basal_feats[i][:sp,:])
        x_test = np.concatenate([x_test,sng_basal_feats[i][sp:,:]])
        y_test = y_test + [-1] * len(sng_basal_feats[i][sp:,:])

In [12]:
# model = RandomForestClassifier(n_estimators=20,max_depth=20)
model = SVC(kernel='rbf', C=1000)
model = model.fit(x_train,y_train)

In [13]:
# print('Accuracy = ', model.score(x_train,y_train))
# print('Accuracy = ', model.score(x_test,y_test))

### Inference on control group

In [14]:
# Cap train
x_train = np.zeros([0,49])
y_train = []
for i in range(len(cap_basal_feats)):
    feat = cap_basal_feats[i]
    feat = feat[bshuffles[i]]
    sp = len(feat)//2
    x_train = np.concatenate([x_train,feat[:sp,:]])
    y_train = y_train + [1] * len(feat[:sp,:])
print('Accuracy = ', model.score(x_train,y_train))
x_train = np.zeros([0,49])
y_train = []
for i in range(len(cap_treat_feats)):
    feat = cap_treat_feats[i]
    feat = feat[tshuffles[i]]
    sp = len(feat)//2
    x_train = np.concatenate([x_train,feat[:sp,:]])
    y_train = y_train + [1] * len(feat[:sp,:])
print('Accuracy = ', model.score(x_train,y_train))
# Cap test
x_test = np.zeros([0,49])
y_test = []
for i in range(len(cap_basal_feats)):
    feat = cap_basal_feats[i]
    feat = feat[bshuffles[i]]
    sp = len(feat)//2
    x_test = np.concatenate([x_test,feat[sp:,:]])
    y_test = y_test + [1] * len(feat[sp:,:])
print('Accuracy = ', model.score(x_test,y_test))
x_test = np.zeros([0,49])
y_test = []
for i in range(len(cap_basal_feats)):
    feat = cap_treat_feats[i]
    feat = feat[tshuffles[i]]
    sp = len(feat)//2
    x_test = np.concatenate([x_test,feat[sp:,:]])
    y_test = y_test + [1] * len(feat[sp:,:])
print('Accuracy = ', model.score(x_test,y_test))
# RO
x_test = np.zeros([0,49])
y_test = []
for i in range(len(ro_basal_feats)):
    x_test = np.concatenate([x_test,ro_basal_feats[i]])
    y_test = y_test + [1] * len(ro_basal_feats[i])
print('Accuracy = ', model.score(x_test,y_test))
x_test = np.zeros([0,49])
y_test = []
for i in range(len(ro_basal_feats)):
    x_test = np.concatenate([x_test,ro_treat_feats[i]])
    y_test = y_test + [1] * len(ro_treat_feats[i])
print('Accuracy = ', model.score(x_test,y_test))
# Sng
x_test = np.zeros([0,49])
y_test = []
for i in range(len(sng_basal_feats)):
    x_test = np.concatenate([x_test,sng_basal_feats[i]])
    y_test = y_test + [1] * len(sng_basal_feats[i])
print('Accuracy = ', model.score(x_test,y_test))
x_test = np.zeros([0,49])
y_test = []
for i in range(len(sng_basal_feats)):
    x_test = np.concatenate([x_test,sng_treat_feats[i]])
    y_test = y_test + [1] * len(sng_treat_feats[i])
print('Accuracy = ', model.score(x_test,y_test))

Accuracy =  0.08920641050395829
Accuracy =  0.741994932043308
Accuracy =  0.1247104247104247
Accuracy =  0.6849252013808976
Accuracy =  0.14216867469879518
Accuracy =  0.37731958762886597
Accuracy =  0.19372234513274336
Accuracy =  0.4069492703266157


In [ ]:
# no pose select
Accuracy =  0.1263572105224805
Accuracy =  0.6791909960226792
Accuracy =  0.17070861554147548
Accuracy =  0.5318393234672304
Accuracy =  0.15990138989246397
Accuracy =  0.3845990666100976
Accuracy =  0.16797344949180668
Accuracy =  0.3458788268036823
# with pose select


### inference on old control groups

In [14]:
cap_basal_feats = []
cap_treat_feats = []
sng_basal_feats = []
sng_treat_feats = []
ro_basal_feats = []
ro_treat_feats = []
root = r'./feat_old/'
files = os.listdir(root)
for file in files:
    [feat,labels] = joblib.load(root+file)
    feat = np.array(feat)
    labels = labels[0]
    if file.find('Cap')!=-1:
        if file.find('basal')!=-1:
            cap_basal_feats.append(select_pose(feat,labels))
        if file.find('treat')!=-1:
            cap_treat_feats.append(select_pose(feat,labels))
    elif file.find('pH5.2')!=-1:
        if file.find('basal')!=-1:
            sng_basal_feats.append(select_pose(feat,labels))
        if file.find('treat')!=-1:
            sng_treat_feats.append(select_pose(feat,labels))
    else:
        if file.find('basal')!=-1:
            ro_basal_feats.append(select_pose(feat,labels))
        if file.find('treat')!=-1:
            ro_treat_feats.append(select_pose(feat,labels))

In [15]:
# Cap train
x_train = np.zeros([0,49])
y_train = []
for i in range(len(cap_basal_feats)):
    sp = len(cap_basal_feats[i])//2
    x_train = np.concatenate([x_train,cap_basal_feats[i][:sp,:]])
    y_train = y_train + [-1] * len(cap_basal_feats[i][:sp,:])
print('Accuracy = ', model.score(x_test,y_test))
x_test = np.zeros([0,49])
y_test = []
for i in range(len(cap_basal_feats)):
    sp = len(cap_treat_feats[i])//2
    x_train = np.concatenate([x_train,cap_treat_feats[i][:sp,:]])
    y_train = y_train + [1] * len(cap_treat_feats[i][:sp,:])
print('Accuracy = ', model.score(x_train,y_train))
# Cap test
x_test = np.zeros([0,49])
y_test = []
for i in range(len(cap_basal_feats)):
    x_test = np.concatenate([x_test,cap_basal_feats[i]])
    y_test = y_test + [1] * len(cap_basal_feats[i])
print('Accuracy = ', model.score(x_test,y_test))
x_test = np.zeros([0,49])
y_test = []
for i in range(len(cap_basal_feats)):
    x_test = np.concatenate([x_test,cap_treat_feats[i]])
    y_test = y_test + [1] * len(cap_treat_feats[i])
print('Accuracy = ', model.score(x_test,y_test))
# RO
x_test = np.zeros([0,49])
y_test = []
for i in range(len(ro_basal_feats)):
    x_test = np.concatenate([x_test,ro_basal_feats[i]])
    y_test = y_test + [1] * len(ro_basal_feats[i])
print('Accuracy = ', model.score(x_test,y_test))
x_test = np.zeros([0,49])
y_test = []
for i in range(len(ro_basal_feats)):
    x_test = np.concatenate([x_test,ro_treat_feats[i]])
    y_test = y_test + [1] * len(ro_treat_feats[i])
print('Accuracy = ', model.score(x_test,y_test))
# Sng
x_test = np.zeros([0,49])
y_test = []
for i in range(len(sng_basal_feats)):
    x_test = np.concatenate([x_test,sng_basal_feats[i]])
    y_test = y_test + [1] * len(sng_basal_feats[i])
print('Accuracy = ', model.score(x_test,y_test))
x_test = np.zeros([0,49])
y_test = []
for i in range(len(sng_basal_feats)):
    x_test = np.concatenate([x_test,sng_treat_feats[i]])
    y_test = y_test + [1] * len(sng_treat_feats[i])
print('Accuracy = ', model.score(x_test,y_test))

Accuracy =  0.18498615860609022
Accuracy =  0.7138666145120282
Accuracy =  0.06795160382667417
Accuracy =  0.1956452129362792
Accuracy =  0.061069773971641164
Accuracy =  0.2849978194505015
Accuracy =  0.086532475617348
Accuracy =  0.18620958317101674
